In [1]:
import numpy
import matplotlib.pyplot as plt
from PIL import Image
import sklearn

In [2]:
def _get_image_as_array(path: str):
    im = Image.open(path)
    im = im.convert(mode="L")
    
    arr = numpy.asarray(im)

    return arr


In [3]:
def _transform_complexArray_to_real(i_arr: numpy.ndarray):
    ret = numpy.absolute(i_arr)
    return ret

In [9]:
def _high_pass_filter(arr: numpy.ndarray, radius: float):
    ret = numpy.zeros((arr.shape[0],arr.shape[1]),dtype=complex)
    for i in range(arr.shape[0]):
        for j in range(arr.shape[1]):
            if (i - arr.shape[0]/2)**2 + (j - arr.shape[1]/2)**2  - radius**2 < 0:
            #if i > arr.shape[0]*radius and j > arr.shape[1]*radius:
            #if (i - arr.shape[0]/2)**2 + (j - arr.shape[1]/2)**2 - radius**2 > 0:
                ret[i][j] = arr[i][j]
            else:
                ret[i][j] = 0
    return ret

def _transform_circular(arr: numpy.ndarray):
    ret = numpy.zeros((arr.shape[0], arr.shape[1]))
    for i in range(arr.shape[0]):
        for j in range(arr.shape[1]):
            if (i - arr.shape[0]/2)**2 + (j - arr.shape[1]/2)**2 -(arr.shape[0]/2)**2 <= 0:
                ret[i][j] = arr[i][j]
            else:
                ret[i][j] = 0
    return ret



In [7]:
def _fft_edge_detector(path: str, radius: float, show: bool):
    image_arr = _get_image_as_array(path)
    #plt.figure("original")
    #plt.imshow(image_arr, interpolation='none')
    #plt.title("Original image")

    image_fft = numpy.fft.fft2(image_arr)
    #plt.figure("dft of image")
    #plt.imshow(_transform_complexArray_to_real(image_fft), interpolation='none')
    #plt.title("dft of image")
    if show:
        im = Image.fromarray(_transform_complexArray_to_real(image_fft))
        im.show()

    fft_filtered = _high_pass_filter(image_fft, radius)
    #plt.figure("filtered dft")
    #plt.imshow(_transform_complexArray_to_real(fft_filtered), interpolation='none')
    #plt.title("filtered dft")
    if show:
        im = Image.fromarray(_transform_complexArray_to_real(fft_filtered))
        im.show()

    filtered_Image = numpy.fft.ifft2(fft_filtered)
    #plt.figure(radius)
    #plt.imshow(_transform_complexArray_to_real(filtered_Image), interpolation='none')
    #plt.title(f"Edge detected {radius}")
    if show:
        im = Image.fromarray(_transform_complexArray_to_real(filtered_Image))
        im.show()
    return _transform_circular( _transform_complexArray_to_real(filtered_Image))



In [13]:
import sklearn
import os

In [ ]:
import zipfile
import os

current_dir = os.getcwd()

data_zip = current_dir + "/data.zip"
data_path = current_dir + "/data"

isColab = False

if isColab:
    os.mkdir(current_dir + "/savs")
    with zipfile.ZipFile(data_zip,"r") as zip:
        zip.extractall(data_path)

In [8]:
X = []
Y = []

TRAIN_PATH = "data/train/"

cls  = os.listdir(TRAIN_PATH)

for i in range(len(cls)):
    imgs = os.listdir(TRAIN_PATH+cls[i])
    for img in imgs:
        path = TRAIN_PATH + cls[i] + "/" + img
        im = _fft_edge_detector(path, 30, False)
        X.append([im.flatten()])
        Y.append(i)


    

In [ ]:
X = numpy.concatenate(X)
Y = numpy.asarray(Y)

In [24]:
from sklearn.decomposition import PCA

pca = PCA(.90)
pca.fit(X,Y)
X_new = pca.transform(X)

In [25]:
numpy.save('savs/X_pca_0.9', X_new)
numpy.save('savs/Y',Y)

In [26]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

clf = make_pipeline(StandardScaler(), SVC())
clf.fit(X_new, Y)

In [21]:

TEST_PATH = "data/validation/"

X_test = []
Y_test = []

cls  = os.listdir(TEST_PATH)

for i in range(len(cls)):
    imgs = os.listdir(TEST_PATH+cls[i])
    for img in imgs:
        path = TEST_PATH + cls[i] + "/" + img
        im = _fft_edge_detector(path, 30, False)
        X_test.append([im.flatten()])
        Y_test.append(i)

X_test = numpy.concatenate(X_test)
Y_test = numpy.asarray(Y_test)

X_test = pca.transform(X_test)

Y_pred = clf.predict(X_test)

numpy.save('savs/X_test_pca_0.9', X_test)
numpy.save('savs/Y_test',Y_test)
numpy.save('savs/Y_pred', Y_pred)

[5]


In [ ]:
from sklearn.metrics import accuracy_score

p = accuracy_score(Y_test, Y_pred)

In [22]:
import pickle
sav_non_pca = 'savs/svm_classifier_with_pca'
pickle.dump(clf,open(sav_non_pca,'wb'))

sav_pca = 'savs/pca_0.9'
pickle.dump(pca, open(sav_pca, 'wb'))